In [ ]:
%matplotlib inline
%matplotlib notebook

import seaborn as sns
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import sys
import gnupg
from rich import print
from rich.panel import Panel
from rich.text import Text
from rich.prompt import Prompt
import pandas as pd
import glob
import os
import shutil
import tarfile
import csv
from DictsAndLists import lista, listaTCA
from pandasql import sqldf

def decrypt_file(file, passphrase, output_file):
    gpg = gnupg.GPG(binary='C:/Program Files (x86)/GnuPG/bin/gpg.exe')
    with open(file, "rb") as f:
        status = gpg.decrypt_file(f, passphrase=passphrase, output=output_file)

    if status.ok:
        print("OK")
    else:
        print(status.stderr)

    print("Buscando paquete...")
    crypto = glob.glob("*.pgp")


def p30stat(paquete):
    # Abrimos el GZ en python para manosearlo por dentro.
    try:
        TarP3 = tarfile.open(paquete, "r:gz")
    except Exception as e:
        print(f"Hubo un error leyendo el archivo de vuelta:{str(e)}")
        Prompt.ask("\nPresione [red][ENTER][/red] para volver al menu.")
        return 0
    print(f"Encontrado: {paquete}")
    p30={}
    # Buscamos los archivos dentro del GZ.
    print("Leyendo vuelta SyNTIS.")
    # Empezamos con el dict "lista"
    for file in lista:
        try:
            filename = [name for name in TarP3.getnames()
                        if file["Nombre"] in name]
            txt = TarP3.extractfile(filename[0])
            df = pd.read_csv(
                txt,
                sep="\t",
                encoding="ansi",
                dtype=file["dtype"],
                names=file["dtype"].keys(),
                header=0,
                index_col=0
            )
            p30[file["Nombre"]] = df
            # print(f"Leyendo {file['Nombre']}")
        except IndexError:
            print(f"'\n{file['Nombre']} no esta en la vuelta.")
            continue
    # Continuamos con el dict "listaTCA"
    for file in listaTCA:
        try:
            filename = [name for name in TarP3.getnames()
                        if file["Nombre"] in name]
            txt = TarP3.extractfile(filename[0])
            df = pd.read_csv(
                txt, sep="\t", encoding="ansi", dtype=file["dtype"],index_col=0)
            # print(f"Leyendo {file['Nombre']}")
            df.insert(loc=0, column="tabla", value=file["Nombre"])
            p30[file["Nombre"]] = df
        except IndexError:
            print(f"\n{file['Nombre']} no esta en la vuelta.\n")
            continue
    return p30
    # Cerramos el GZ para que no haya error al moverlo.
    TarP3.close()

paquete30=p30stat('Vuelta_EfectoresPaquete_30.tar.gz')
p30 = pd.read_fwf('EfectoresPaquete_30.txt',encoding='ANSI',widths=(15,9,50,50,8,1,2,10,20,40,6,5,5,5,5,20,20,30,8,20,20,30,30,50,50,15,50,50,30,50,50,11,5,70,6,2,12,12))
# cuits= pd.read_csv("cuits.txt",sep="\t")


#### GUARDAMOS LOS DICT EN VARIABLES USABLES

In [ ]:

var = paquete30['B00']
dat = paquete30['DATOS']
pnc = paquete30['PNC']
jub = paquete30['JUBILACIONES_PENSIONES']
dep = paquete30['EMPLEO_DEPENDIENTE']
asg = paquete30['ASIGNACIONES']
fal = paquete30['FALLECIDOS']
jur = paquete30['PERSONAS_JURIDICAS']
rub = paquete30['RUBPS']
aut = paquete30['PADRON_AUTOMOTORES']
avi = paquete30['AERONAVES']
bar = paquete30['EMBARCACIONES']
inm = paquete30['INMUEBLES']

In [ ]:
crucepaalexis = sqldf("""
                SELECT *
                p.dDocNroD
                FROM      var d 
                LEFT JOIN p30 p on p.dDocNroD   = d.[ NDOC  ]
                inner JOIN (select distinct id_persona from jub) j on j.id_persona = d.[id_persona ]
                    """)

In [ ]:
crucepaalexis.to_excel()


In [ ]:
# activos = sqldf("""
#                 SELECT 
#                 [ PROV_ACTIVIDAD  ] as PROVINCIA,
#                 COUNT(DISTINCT d.[id_persona ]) as EFECTORES,
#                 SUM(case when c.cuit is not null then 1 else 0 end) as EFECTORES_activos,
#                 COUNT(DISTINCT p.id_persona) as PNC, 
#                 SUM(case when c.cuit is not null and p.id_persona is not null then 1 else 0 end) as PNC_activos,
#                 COUNT(DISTINCT j.id_persona) as JUB_PEN,
#                 SUM(case when c.cuit is not null and j.id_persona is not null then 1 else 0 end) as JUB_PEN_activos,
#                 COUNT(DISTINCT e.id_persona) as EMPLEO_DEP,
#                 SUM(case when c.cuit is not null and e.id_persona is not null then 1 else 0 end) as EMPLEO_DEP_activos,
#                 COUNT(DISTINCT a.id_persona) as ASIG_FAM,
#                 SUM(case when c.cuit is not null and a.id_persona is not null then 1 else 0 end) as ASIG_FAM_DEP_activos,
#                 COUNT(DISTINCT f.id_persona) as FALLECIDOS,
#                 SUM(case when c.cuit is not null and f.id_persona is not null then 1 else 0 end) as FALLECIDOS_activos,
#                 COUNT(DISTINCT h.id_persona) as JURIDICAS,
#                 SUM(case when c.cuit is not null and h.id_persona is not null then 1 else 0 end) as JURIDICAS_activos,
#                 COUNT(DISTINCT r.id_persona) as RUBPS,
#                 SUM(case when c.cuit is not null and r.id_persona is not null then 1 else 0 end) as RUBPS_activos
#                 FROM      var d 
#                 LEFT JOIN cuits c on c.cuit       = d.[ CUIT  ]
#                 LEFT JOIN pnc   p on p.id_persona = d.[id_persona ]
#                 LEFT JOIN dat   t on t.id_persona = d.[id_persona ]
#                 LEFT JOIN jub   j on j.id_persona = d.[id_persona ]
#                 LEFT JOIN dep   e on e.id_persona = d.[id_persona ]
#                 LEFT JOIN asg   a on a.id_persona = d.[id_persona ]
#                 LEFT JOIN fal   f on f.id_persona = d.[id_persona ]
#                 LEFT JOIN jur   h on h.id_persona = d.[id_persona ]
#                 LEFT JOIN rub   r on r.id_persona = d.[id_persona ]
#                 GROUP BY [ PROV_ACTIVIDAD  ]
#               """)
activos = sqldf("""
                SELECT 
                [ PROV_ACTIVIDAD  ] as PROVINCIA,
                COUNT(DISTINCT d.[id_persona ])                                                  as EFECTORES,
                SUM(case when c.cuit is not null then 1 else 0 end)                              as EFECTORES_activos,
                COUNT(DISTINCT p.id_persona)                                                     as PNC, 
                SUM(case when c.cuit is not null and p.id_persona is not null then 1 else 0 end) as PNC_activos,
                COUNT(DISTINCT j.id_persona)                                                     as JUB_PEN,
                SUM(case when c.cuit is not null and j.id_persona is not null then 1 else 0 end) as JUB_PEN_activos,
                COUNT(DISTINCT e.id_persona)                                                     as EMPLEO_DEP,
                SUM(case when c.cuit is not null and e.id_persona is not null then 1 else 0 end) as EMPLEO_DEP_activos,
                COUNT(DISTINCT a.id_persona)                                                     as ASIG_FAM,
                SUM(case when c.cuit is not null and a.id_persona is not null then 1 else 0 end) as ASIG_FAM_activos,
                COUNT(DISTINCT f.id_persona)                                                     as FALLECIDOS,
                SUM(case when c.cuit is not null and f.id_persona is not null then 1 else 0 end) as FALLECIDOS_activos,
                COUNT(DISTINCT h.id_persona)                                                     as JURIDICAS,
                SUM(case when c.cuit is not null and h.id_persona is not null then 1 else 0 end) as JURIDICAS_activos,
                COUNT(DISTINCT r.id_persona)                                                     as RUBPS,
                SUM(case when c.cuit is not null and r.id_persona is not null then 1 else 0 end) as RUBPS_activos
                FROM      var d 
                LEFT JOIN cuits c on c.cuit       = d.[ CUIT  ]
                LEFT JOIN (select distinct id_persona from pnc) p on p.id_persona = d.[id_persona ]
                LEFT JOIN (select distinct id_persona from jub) j on j.id_persona = d.[id_persona ]
                LEFT JOIN (select distinct id_persona from dep) e on e.id_persona = d.[id_persona ]
                LEFT JOIN (select distinct id_persona from asg) a on a.id_persona = d.[id_persona ]
                LEFT JOIN (select distinct id_persona from fal) f on f.id_persona = d.[id_persona ]
                LEFT JOIN (select distinct id_persona from jur) h on h.id_persona = d.[id_persona ]
                LEFT JOIN (select distinct id_persona from rub) r on r.id_persona = d.[id_persona ]
                GROUP BY [ PROV_ACTIVIDAD  ]
              """)
activos.set_index('PROVINCIA', inplace=True)
activos.loc['TOTAL']= activos.sum(numeric_only=True)
activos

#### PREPARAMOS EL EXCEL

In [ ]:
# preparamos el excel
writer = pd.ExcelWriter(
    'Resumen_P30.xlsx', engine="xlsxwriter"
)

#### PRIMERO SACAMOS TODOS LOS STATS GENERALES POR PROVINCIA

In [ ]:
stats_prov = sqldf("""
                SELECT 
                [ PROV_ACTIVIDAD  ] as PROVINCIA,
                [ LOCALIDAD  ] as LOCALIDAD,
                t.sexo,
                COUNT(DISTINCT d.[id_persona ]) as EFECTORES,
                COUNT(DISTINCT p.id_persona)    as PNC, 
                COUNT(DISTINCT j.id_persona)    as JUB_PEN,
                COUNT(DISTINCT e.id_persona)    as EMPLEO_DEP,
                COUNT(DISTINCT a.id_persona)    as ASIG_FAM,
                COUNT(DISTINCT f.id_persona)    as FALLECIDOS,
                COUNT(DISTINCT h.id_persona)    as JURIDICAS,
                COUNT(DISTINCT r.id_persona)    as RUBPS
                FROM      var d 
                LEFT JOIN pnc p on p.id_persona = d.[id_persona ]
                LEFT JOIN dat t on t.id_persona = d.[id_persona ]
                LEFT JOIN jub j on j.id_persona = d.[id_persona ]
                LEFT JOIN dep e on e.id_persona = d.[id_persona ]
                LEFT JOIN asg a on a.id_persona = d.[id_persona ]
                LEFT JOIN fal f on f.id_persona = d.[id_persona ]
                LEFT JOIN jur h on h.id_persona = d.[id_persona ]
                LEFT JOIN rub r on r.id_persona = d.[id_persona ]
                GROUP BY [ PROV_ACTIVIDAD  ],[ LOCALIDAD  ],t.sexo
              """)
stats_prov.set_index('PROVINCIA', inplace=True)
stats_prov.loc['TOTAL']= stats_prov.sum(numeric_only=True)
stats_prov.to_csv('Stats_por_provincia.txt', sep ='\t')

#### EVALUACION MUEBLES

In [ ]:
muebles= sqldf("""
                select * from aut
                where ID_PERSONA IN ( 
                select dat.id_persona
                from dat inner join 
                aut on aut.ID_PERSONA=dat.id_persona
                GROUP BY dat.id_persona
                HAVING COUNT(DOMINIO)>2)
                ORDER BY ID_PERSONA""")
muebles.set_index('ID_PERSONA', inplace=True)

#### EVALUACION INMUEBLES

In [ ]:
inmuebles= sqldf("""
                select * from inm
                where ID_PERSONA IN ( 
                select id_persona
                from inm
                GROUP BY id_persona
                HAVING COUNT(ID_PERSONA)>1)
                ORDER BY ID_PERSONA""")
inmuebles.set_index('ID_PERSONA', inplace=True)

#### ARMAMOS EL DF DE LOS EVALUADOS

In [ ]:
eval = sqldf("""
            SELECT
            distinct(dat.id_persona),
            dat.cuit,
            dat.ndoc,
            dat.deno,
            case when dat.sexo=1 then 'M' else 'F' end as sexo,
            dat.fnac,
            dat.grado_confiab,
            case when inm.id_persona is not null then 'Si' else 'No' end as Inmuebles,
            case when aut.id_persona is not null then 'Si' else 'No' end as Muebles,
            case when dep.base=105 then 'Si' else 'No' end as Servicio_Domestico,
            case when dep.monto > 45000 then 'Si' else 'No' end as Dependiente,
            case when jur.id_persona is not null then 'Si' else 'No' end as Juridicas,
            case when jub.monto > 45000 then 'Si' else 'No' end as Jub_pen,
            case when fal.id_persona is not null then 'Si' else 'No' end as Fallecido
            from dat
            left join muebles aut on aut.ID_PERSONA = dat.id_persona
            left join inmuebles inm on inm.ID_PERSONA = dat.id_persona
            left join dep on dep.ID_PERSONA = dat.id_persona
            left join jub on jub.ID_PERSONA = dat.id_persona
            left join jur on jur.ID_PERSONA = dat.id_persona
            left join fal on fal.ID_PERSONA = dat.id_persona
            where 
            Inmuebles ='Si' 
            or Muebles ='Si' 
            or Servicio_Domestico ='Si' 
            or Dependiente ='Si'
            or Juridicas ='Si'
            or Jub_pen ='Si'
            or Fallecido='Si'
                """)

#### ARMAMOS EL EXCEL DE SALIDA

In [ ]:
stats_prov.to_excel(writer, sheet_name="Stats globales")
eval.to_excel(writer, sheet_name="Efectores fuera del marco legal")
muebles.to_excel(writer, sheet_name="Detalle Muebles")
inmuebles.to_excel(writer, sheet_name="Detalle Inmuebles")
writer.save()